# LIBRAIRIES IMPORTATION

In [2]:
import pandas as pd
import numpy as np


import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import io
pio.renderers.default = "vscode"


import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
from PIL import Image
pal = sns.color_palette()

from sklearn.preprocessing import  StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) # to avoid deprecation warnings

 # File reading and basic exploration 

In [28]:
# Import dataset
print("Loading dataset...")
dataset = pd.read_csv("./assets/flights_delays_from_jan2013_to_ jan2023.csv")  # sep = ";"
pd.set_option('display.max_columns', None)
print("...Done.")
print()

Loading dataset...
...Done.



In [29]:
# Basic stats
print("Number of rows : {}".format(dataset.shape[0]))
print()

print("Display of dataset: ")
display(dataset.head())
print()

print("Basics statistics: ")
data_desc = dataset.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")
display(100*dataset.isnull().sum()/dataset.shape[0])

Number of rows : 167110

Display of dataset: 


,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2023,1,9E,Endeavor Air Inc.,ABE,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",14.0,1.0,0.00,1.00,0.00,0.0,0.00,0.0,0.0,648.0,0.0,647.0,1.0,0.0,0.0
1,2023,1,9E,Endeavor Air Inc.,ABY,"Albany, GA: Southwest Georgia Regional",82.0,10.0,6.06,1.72,0.47,0.0,1.74,0.0,0.0,1800.0,1393.0,272.0,56.0,0.0,79.0
2,2023,1,9E,Endeavor Air Inc.,AEX,"Alexandria, LA: Alexandria International",60.0,8.0,3.54,1.59,2.71,0.0,0.16,2.0,4.0,484.0,176.0,184.0,113.0,0.0,11.0
3,2023,1,9E,Endeavor Air Inc.,AGS,"Augusta, GA: Augusta Regional at Bush Field",26.0,6.0,3.40,1.20,0.64,0.0,0.76,0.0,0.0,395.0,119.0,167.0,30.0,0.0,79.0
4,2023,1,9E,Endeavor Air Inc.,ALB,"Albany, NY: Albany International",109.0,30.0,8.08,0.61,9.80,0.0,11.52,3.0,0.0,1591.0,402.0,22.0,333.0,0.0,834.0



Basics statistics: 


,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
count,167110.000000,167110.000000,167110,167110,167110,167110,166876.000000,166669.000000,166876.000000,166876.000000,166876.000000,166876.000000,166876.000000,166876.000000,166876.000000,166876.000000,166876.000000,166876.000000,166876.000000,166876.000000,166876.000000
unique,NaN,NaN,21,24,396,417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,OO,SkyWest Airlines Inc.,AUS,"Austin, TX: Austin - Bergstrom International",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,24534,24534,1512,1512,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,2017.962300,6.489019,NaN,NaN,NaN,NaN,365.421600,66.427500,20.537636,2.211416,19.683589,0.151449,23.761061,7.549756,0.857900,4170.705326,1395.480602,214.902365,929.705224,7.044206,1623.568596
std,2.944419,3.488475,NaN,NaN,NaN,NaN,997.753344,178.670761,49.076928,7.162642,62.784311,0.693100,71.894932,43.923477,3.742051,12240.947749,3995.171204,795.025358,3476.399194,40.384665,5018.119222
min,2013.000000,1.000000,NaN,NaN,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2015.000000,3.000000,NaN,NaN,NaN,NaN,51.000000,6.000000,2.150000,0.000000,1.020000,0.000000,1.230000,0.000000,0.000000,333.000000,110.000000,0.000000,35.000000,0.000000,65.000000
50%,2018.000000,7.000000,NaN,NaN,NaN,NaN,102.000000,17.000000,6.420000,0.370000,4.000000,0.000000,5.000000,1.000000,0.000000,1016.000000,374.000000,17.000000,149.000000,0.000000,320.000000
75%,2021.000000,10.000000,NaN,NaN,NaN,NaN,253.000000,48.000000,17.330000,1.820000,11.990000,0.000000,15.390000,4.000000,1.000000,2879.000000,1099.000000,142.250000,484.000000,0.000000,1067.000000



Percentage of missing values: 


year                   0.000000
month                  0.000000
carrier                0.000000
carrier_name           0.000000
airport                0.000000
airport_name           0.000000
arr_flights            0.140028
arr_del15              0.263898
carrier_ct             0.140028
weather_ct             0.140028
nas_ct                 0.140028
security_ct            0.140028
late_aircraft_ct       0.140028
arr_cancelled          0.140028
arr_diverted           0.140028
arr_delay              0.140028
carrier_delay          0.140028
weather_delay          0.140028
nas_delay              0.140028
security_delay         0.140028
late_aircraft_delay    0.140028
dtype: float64

In [30]:
print('List of columns:')
list_columns = list(dataset.columns)
print(list(dataset.columns))
print(f'Total: {len(list(dataset.columns))} columns')


List of columns:
['year', 'month', 'carrier', 'carrier_name', 'airport', 'airport_name', 'arr_flights', 'arr_del15', 'carrier_ct', 'weather_ct', 'nas_ct', 'security_ct', 'late_aircraft_ct', 'arr_cancelled', 'arr_diverted', 'arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']
Total: 21 columns
